## Comparison

#### Preambule

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path

#### Data reading

In [2]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
xr_2030 = xr.open_dataset("K:/ECEMF/T5.2/xr_alloc_2030.nc")

In [3]:
df_berg = pd.read_excel(Path("X:/user/dekkerm/Data/") / "Berg_paper" / "Berg2019_data.xlsx", sheet_name = "Emission pathways")
df_berg2 = df_berg[(df_berg.Scenario == 'Gt400') & (df_berg.Category == 'Emissions|Kyoto Gases')].drop(columns=['Scenario', 'Fixed variable', 'Category', 'Unnamed: 10',  'Unnamed: 12', 'Region #'])
df_berg2 = df_berg2.rename({'Sensitivity case': "Parameters", 'Year': 'Time'}, axis=1)
approaches = np.array(df_berg2.Approach)
approaches[approaches == 'IEPC'] = 'PC'
df_berg2['Approach'] = approaches
df_berg2 = df_berg2.melt(id_vars=["Approach", "Region", "Parameters", "Time"], var_name="Range", value_name="Value")
df_berg2 = df_berg2.set_index(['Approach', "Region", "Parameters", "Range", "Time"])
xr_berg = df_berg2.to_xarray()

In [4]:
df_rob = pd.read_excel(Path("X:/user/dekkerm/Data/") / "Robiou_paper" / "Robiou2016_15data.xlsx", sheet_name = "1.5°C National allocations")
df_rob2 = df_rob.drop(columns=['Country names', 'INDC Low assessment', 'INDC average assessment', 'INDC High assessment', 2025, '2025.1', '2030.1', 2040, '2040.1', 2050, '2050.1', 'Peaking year', 'First net-zero allocation year', 'Budget 2010-2050', 'Budget 2010-2100'])
df_rob2 = df_rob2.rename({'Country code ISO Alpha-3': 'Region', "approaches names": "Approach", 2030:'Val2030wrt2010'}, axis=1)
approaches = np.array(df_rob2.Approach)
approaches[approaches == 'CER'] = 'GF'
approaches[approaches == 'EPC'] = 'PCC'
approaches[approaches == 'CAP'] = 'AP'
approaches[approaches == 'CPC'] = 'ECPC'
df_rob2['Approach'] = approaches
df_rob2 = df_rob2.set_index(['Approach', 'Region'])
xr_rob = df_rob2.to_xarray()

In [5]:
df_raj = pd.read_csv(Path("X:/user/dekkerm/Data/") / "Rajamani_paper" / "Data.csv")
df_raj2 = df_raj[df_raj.keys()[[1, -1, -2]]]
df_raj2 = df_raj2.rename({'ISO3': 'Region', 'Full literature range maximum 2030 [% of 2010]': 'Maximum_2030wrt2010', 'Full literature range minimum 2030 [% of 2010]' : 'Minimum_2030wrt2010'}, axis=1)
df_raj3 = df_raj2.melt(id_vars=["Region"], var_name="Variable", value_name="Value")
df_raj3 = df_raj3.set_index(['Variable', 'Region'])
xr_raj = df_raj3.to_xarray()

In [56]:
all_countries_iso = np.load(Path("K:/ECEMF/T5.2/") / "all_countries.npy", allow_pickle=True)

#### Comparison

Check the minima of the range within GF, PCC, GDR and AP for any given country (cty):

In [75]:
vary = ['GF', 'PCC', 'GDR', 'AP']
cty = 'IND'

# Own work
val_2005 = float(xr_dataread.GHG_hist.sel(Time=2010, Region=cty))
val_year = np.array(xr_2030.sel(Temperature=1.5, Risk=0.5, NegEmis=0.5, Timing='Immediate', NonCO2red=0.5, Region=cty, Convergence_year=2050).mean(dim=['Scenario'])[vary].to_array())
print('Own work:',np.round(np.min((val_year - val_2005)/val_2005), 3))

# Berg
try:
    val_2010 = np.array(xr_berg.sel(Approach=vary, Range='default', Region=cty, Time=2010).mean(dim='Parameters').Value)
    val_year = np.array(xr_berg.sel(Approach=vary, Range='default', Region=cty, Time=2030).mean(dim='Parameters').Value)
    print('Berg:',np.round(np.min((val_year - val_2010)/val_2010), 3))
except:
    3

# Robiou
val_yearwrt2010 = np.array(xr_rob.sel(Approach=vary, Region=cty).Val2030wrt2010/100)
print('Robiou (excl LULUCF):',np.round(np.min(val_yearwrt2010), 3))

# Raj
val = 1-float(xr_raj.sel(Variable = 'Minimum_2030wrt2010', Region=cty).Value)/100
print('Rajamani (excl LULUCF):',np.round(-val, 3))

Own work: -0.188
Berg: -0.315
Robiou (excl LULUCF): -0.33
Rajamani (excl LULUCF): -0.188


Also check the maximum allocation (all in 2030 wrt 2010):

In [74]:
vary = ['GF', 'PCC', 'GDR', 'AP']
cty = 'NLD'

# Own work
val_2005 = float(xr_dataread.GHG_hist.sel(Time=2010, Region=cty))
val_year = np.array(xr_2030.sel(Temperature=1.5, Risk=0.5, NegEmis=0.5, Timing='Immediate', NonCO2red=0.5, Region=cty, Convergence_year=2050).mean(dim=['Scenario'])[vary].to_array()) #Timing='Immediate',  , NonCO2red=0.5
print('Own work:',np.round(np.max((val_year - val_2005)/val_2005), 3))

# Berg
try:
    val_2010 = np.array(xr_berg.sel(Approach=vary, Range='default', Region=cty, Time=2010).mean(dim='Parameters').Value)
    val_year = np.array(xr_berg.sel(Approach=vary, Range='default', Region=cty, Time=2030).mean(dim='Parameters').Value)
    print('Berg:',np.round(np.max((val_year - val_2010)/val_2010), 3))
except:
    3

# Robiou
val_yearwrt2010 = np.array(xr_rob.sel(Approach=vary, Region=cty).Val2030wrt2010/100)
print('Robiou (excl LULUCF):',np.round(np.max(val_yearwrt2010), 3))

# Raj
val = 1-float(xr_raj.sel(Variable = 'Maximum_2030wrt2010', Region=cty).Value)/100
print('Rajamani (excl LULUCF):',np.round(-val, 3))

Own work: -0.53
Robiou (excl LULUCF): -0.33
Rajamani (excl LULUCF): -0.391


Per allocation rule (Rajamani does not provide this):

In [76]:
cty = 'BRA'

for vary in ['GF', 'PCC', 'AP', 'GDR', 'ECPC']:
    print('=========', vary, '=========')
    # Own work
    val_2005 = xr_dataread.GHG_hist.sel(Time=2010, Region=cty)
    val_year = xr_2030.sel(Temperature=1.5, Risk=0.5, NegEmis=0.5, Timing='Immediate', NonCO2red=0.5, Region=cty, Convergence_year=2050).mean(dim=['Scenario'])[vary] #Timing='Immediate',  , NonCO2red=0.5
    print('Own work:',np.round(float((val_year - val_2005)/val_2005), 3))

    # Berg
    try:
        val_2010 = xr_berg.sel(Approach=vary, Range='default', Region=cty, Time=2010).mean(dim='Parameters').Value
        val_year = xr_berg.sel(Approach=vary, Range='default', Region=cty, Time=2030).mean(dim='Parameters').Value
        print('Berg:',np.round(float((val_year - val_2010)/val_2010), 3))
    except:
        3

    # Robiou
    val_yearwrt2010 = float(xr_rob.sel(Approach=vary, Region=cty).Val2030wrt2010/100)
    print('Robiou (excl LULUCF):',np.round(val_yearwrt2010, 3))
    print('')

========= GF =========
Own work: -0.559
Berg: -0.315
Robiou (excl LULUCF): -0.33

========= PCC =========
Own work: -0.581
Berg: -0.439
Robiou (excl LULUCF): -0.29

========= AP =========
Own work: -0.558
Berg: -0.382
Robiou (excl LULUCF): -0.54

========= GDR =========
Own work: -0.449
Berg: -0.388
Robiou (excl LULUCF): -0.28

========= ECPC =========
Own work: -0.747
Robiou (excl LULUCF): -0.39

